# Dataset - Feature Extraction

In this notebook, the dataset used to train the machine learning model is created

In [11]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import time

pd.set_option('mode.chained_assignment', None)

In [12]:
# Constants
data_dir = 'geolife-data/Prepared'
output_file = 'processed_data.csv'
files_to_read = 10
earth_radius = 6367

## Load files

In [13]:
filelist = os.listdir(data_dir)
files_to_read = (len(filelist) if files_to_read>len(filelist) else files_to_read)

data_raw = pd.DataFrame()
for file in filelist[:files_to_read]:
    if os.path.isdir(data_dir + "/" + file):
        continue
    data_raw_temp = traj_df = pd.read_csv(data_dir + "/" + file)
    data_raw = pd.concat([data_raw, data_raw_temp])


## Prepare Data

In [14]:
data_raw = data_raw.rename(columns={"height": "altitude"})\
    .drop(columns = ['days_total', 'date', 'time'])\
    .dropna()

In [15]:
data_raw['record_dt'] = data_raw['record_dt'].astype("datetime64")

In [16]:
def haversine(lat1, lon1, lat2, lon2, earth_radius=6371):
    
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    a = np.sin(np.abs(lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(np.abs(lon2-lon1)/2.0)**2
    c = np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
    return earth_radius * 2 * c

## Process data

In [17]:
users = data_raw['user'].unique()

In [21]:
data = pd.DataFrame()
for user in users[:files_to_read]:
    print("User: " + str(user))
    data_user = data_raw[data_raw.user == user]
    trips = data_user['trans_trip'].unique()
    for trip in trips[:]:
        
        # data_traj consist of all trajectories of one singel trip
        
        data_traj = data_user[data_user.trans_trip == trip]
        data_traj = data_traj.sort_values(by=['record_dt'])
        
        # Distance to next entry in meters
        data_traj['dist'] = haversine(data_traj.latitude, data_traj.longitude, data_traj.latitude.shift(-1), data_traj.longitude.shift(-1)) * 1000
        
        # Time difference to next entry in seconds
        data_traj['time_delta'] = (data_traj.record_dt.shift(-1) - data_traj.record_dt).astype("int64") / (1000000000.0)
        
        # Velocity until next entry in m/s
        data_traj['velocity'] = data_traj['dist'] / (data_traj['time_delta'])
        
        # Acceleration in m/(sˆ2)
        data_traj['acceleration'] = (data_traj['velocity'].shift(-1) - data_traj['velocity']) / (data_traj['time_delta'])
        
        
        # Features 
        
        temp =  pd.DataFrame()

        for i in range(3):
            temp['v' + str(i)] = data_traj.velocity.shift(-i)

        for i in range(3):
            temp['a' + str(i)] = data_traj.acceleration.shift(-i)

        
        
        
        temp['max(a)'] = temp[['a0', 'a1', 'a2']].values.max(1)
        temp['min(a)'] = temp[['a0', 'a1', 'a2']].values.min(1)
        temp['avg(a)'] = (temp.a0 + temp.a1 + temp.a2) / 3
        temp['var(a)'] =   ((temp.a0 - temp['avg(a)'])**2\
                         + (temp.a1 - temp['avg(a)'])**2\
                         + (temp.a2 - temp['avg(a)']**2))/3
        

        
        temp['max(v)'] = temp[['v0', 'v1', 'v2']].values.max(1)
        temp['min(v)'] = temp[['v0', 'v1', 'v2']].values.min(1)
        temp['avg(v)'] = (temp.v0 + temp.v1 + temp.v2) / 3
        temp['var(v)'] =   ((temp.v0 - temp['avg(v)'])**2\
                 + (temp.v1 - temp['avg(v)'])**2\
                 + (temp.v2 - temp['avg(v)']**2))/3
        


        temp['class'] = data_traj['trans_mode']
        temp = temp.dropna()
        
        data = pd.concat([data,temp])
 

User: 106
User: 138
User: 163
User: 82
User: 64
User: 20
User: 89
User: 112
User: 96
User: 128


## Preprocessing

In [42]:
final = data

In [43]:
len(final)

925247

In [44]:
# taxi -> car
final['class'][final['class'] == 'taxi'] = 'car'

used_classes = ['car', 'walk', 'bike', 'bus', 'train']

final = final.drop(final[~final['class'].isin(used_classes)].index)

In [45]:
final = final.drop(final[(final['class'] == 'walk') & ((final['max(v)'] > 7) | (final['max(a)'] >= 3))].index)
final = final.drop(final[(final['class'] == 'bike') & ((final['max(v)'] > 12) | (final['max(a)'] >= 3))].index)
final = final.drop(final[(final['class'] == 'bus') & ((final['max(v)'] > 34) | (final['max(a)'] >= 2))].index)
final = final.drop(final[(final['class'] == 'car') & ((final['max(v)'] > 50) | (final['max(a)'] >= 10))].index)
final = final.drop(final[(final['class'] == 'train') & ((final['max(v)'] > 34) | (final['max(a)'] >= 3))].index)

In [46]:
print(len(final))

483057


In [47]:
final.to_csv(output_file)

In [48]:
final.groupby(by='class').count()

,v0,v1,v2,a0,a1,a2,max(a),min(a),avg(a),var(a),max(v),min(v),avg(v),var(v)
class,,,,,,,,,,,,,,
bike,79307,79307,79307,79307,79307,79307,79307,79307,79307,79307,79307,79307,79307,79307
bus,30903,30903,30903,30903,30903,30903,30903,30903,30903,30903,30903,30903,30903,30903
car,214843,214843,214843,214843,214843,214843,214843,214843,214843,214843,214843,214843,214843,214843
train,12571,12571,12571,12571,12571,12571,12571,12571,12571,12571,12571,12571,12571,12571
walk,145433,145433,145433,145433,145433,145433,145433,145433,145433,145433,145433,145433,145433,145433
